This notebook serves to scrape a large number of neurology-related PDF files from PubMed. 

In [1]:
import os 
import requests 
import time
from bs4 import BeautifulSoup 
import xml.etree.ElementTree as ET
from tqdm import tqdm
import tarfile

In [2]:
tree = ET.parse('pmcquery.xml')
root = tree.getroot() 

In [3]:
id_list = []

for id_element in root.findall(".//IdList/Id"): 
    id_list.append(id_element.text)

print(f'Number of neurology-related PMCIDs from the previous year: {len(id_list)}')

Number of neurology-related PMCIDs from the previous year: 6801


In [4]:
def download_file(url, save_path):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(save_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
        else:
            print(f"Failed to download: {url} (Status code: {response.status_code})")
    except Exception as e:
        print(f"Error downloading {url}: {e}")


def extract_tar_gz(file_path, extract_to_folder):
    try:
        with tarfile.open(file_path, "r:gz") as tar:
            tar.extractall(path=extract_to_folder)
    except Exception as e:
        print(f"Error extracting {file_path}: {e}")        

In [5]:
base_url = 'https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC' # We need only to append the id to this url and get all availible data for the article.
save_folder = 'dfolder'

for id in tqdm(id_list):

    document_base_url = base_url + id
    time.sleep(5)
    try:
        response = requests.get(url=document_base_url)

        if response.status_code == 200:
            root = ET.fromstring(response.text)

            tar_gz_url = None
            
            for link in root.findall(".//link"):
                format_type = link.attrib.get("format")
                href = link.attrib.get("href")
                if format_type == "tgz":
                    tar_gz_url = href[6:]
                    tar_gz_url = "https://" + tar_gz_url
                    break

            if tar_gz_url:

                    file_name = f"{id}.tar.gz"
                    file_path = os.path.join(save_folder, file_name)
                    
                    download_file(tar_gz_url, file_path)
                    extract_tar_gz(file_path, save_folder)

            else:
                print(f"No tar.gz link found for {id}")
    
    except Exception as e:
        print(f"Failed to fetch data for {id} (Status code: {response.status_code})") 


  0%|          | 0/6801 [00:00<?, ?it/s]/tmp/ipykernel_7257/898492519.py:17: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=extract_to_folder)
  0%|          | 30/6801 [04:34<30:47:31, 16.37s/it]

Failed to fetch data for 11488616 (Status code: 200)


  2%|▏         | 110/6801 [17:12<29:56:09, 16.11s/it]

Failed to fetch data for 11397496 (Status code: 200)


  2%|▏         | 127/6801 [19:46<29:45:54, 16.06s/it]

...

100%|█████████▉| 6771/6801 [15:50:59<08:01, 16.05s/it]

Failed to fetch data for 7149322 (Status code: 200)


100%|█████████▉| 6785/6801 [15:53:21<04:27, 16.74s/it]

Failed to fetch data for 8029192 (Status code: 200)


100%|██████████| 6801/6801 [15:55:24<00:00,  8.43s/it]


In [6]:
import os
import shutil

def move_pdfs(source_folder="dfolder", destination_folder="pdfs"):
    # Ensure the destination folder exists
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Walk through the source folder recursively
    for root, _, files in os.walk(source_folder):
        for file in files:
            # Check if the file is a PDF
            if file.lower().endswith(".pdf"):
                # Build full paths for the source and destination
                source_path = os.path.join(root, file)
                destination_path = os.path.join(destination_folder, file)

                # Move the PDF file to the destination folder
                shutil.move(source_path, destination_path)
                print(f"Moved: {source_path} to {destination_path}")

# Call the function
move_pdfs()


Moved: dfolder/PMC8712250/cureus-0013-00000019934.pdf to pdfs/cureus-0013-00000019934.pdf
Moved: dfolder/PMC6914700/fnagi-11-00347.pdf to pdfs/fnagi-11-00347.pdf
Moved: dfolder/PMC10233237/bhad017.pdf to pdfs/bhad017.pdf
Moved: dfolder/PMC10023984/main.pdf to pdfs/main.pdf
Moved: dfolder/PMC11365916/12264_2024_Article_1218.pdf to pdfs/12264_2024_Article_1218.pdf
Moved: dfolder/PMC11365916/12264_2024_1218_MOESM1_ESM.pdf to pdfs/12264_2024_1218_MOESM1_ESM.pdf
Moved: dfolder/PMC7034269/NRR-15-1111_Suppl1.pdf to pdfs/NRR-15-1111_Suppl1.pdf
Moved: dfolder/PMC7034269/NRR-15-1111.pdf to pdfs/NRR-15-1111.pdf
Moved: dfolder/PMC7772800/13195_2020_Article_744.pdf to pdfs/13195_2020_Article_744.pdf
Moved: dfolder/PMC10172615/main.pdf to pdfs/main.pdf
Moved: dfolder/PMC9547798/401_2022_2469_MOESM2_ESM.pdf to pdfs/401_2022_2469_MOESM2_ESM.pdf
Moved: dfolder/PMC9547798/401_2022_Article_2469.pdf to pdfs/401_2022_Article_2469.pdf
Moved: dfolder/PMC9547798/401_2022_2469_MOESM1_ESM.pdf to pdfs/401_2022_2